# Warping tutorial
## D_playing_with_time_origin

##### May 2020
###### Eva Chamorro - Daniel Zitterbart - Julien Bonnel

## 1. Import packages

In [1]:
import os
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
%matplotlib widget
from matplotlib import interactive
from matplotlib.path import Path
from warping_functions import *
from time_frequency_analysis_functions import *
from bbox_select import *
from scipy.signal import hilbert
from ipywidgets import interact, interact_manual
from pts_select import *
import warnings
warnings.filterwarnings('ignore')

## 2. Load simulated signal

In [2]:
data = sio.loadmat(os.getcwd()+ '/sig_pek_for_warp.mat')


'''
    s_t: propagated modes in a Pekeris waveguide with parameters
    c1, c2, rho1, rho2: sound speed / density
        D: depth
        r: range
        zs, zr: source/receiver depth
    s_t_dec: same than s_t, except that time origin has been set for warping
    fs: sampling frequency
    
   
     NB: one can run optional_create_simulated_signal.m to generate another
     simulated signal
'''

# Select variables 
s_t=data['s_t']
fs=data['fs']
s_t_dec=data['s_t_dec']
r=data['r']
c1=data['c1']

### IF YOU CHANGE RANGE, you must change the following variable which
### defines the bounds on the x-axis for all the plots

xlim_plots=[6.5,7.5]

## 3. Plot spectrogram and time origin selection

In [4]:
N=len(s_t[0,:])
NFFT=2048 # FFT size
N_window=31 # sliding window size (need an odd number)

a=np.arange(1,N+1)
a=a[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]
tfr=tfrstft(s_t,a,NFFT,d)
spectro=abs(tfr)**2
    
time=np.arange(0,N)/fs
freq=np.arange(0,NFFT)*fs/NFFT
    
print('This is the spectrogram of a signal propagated in a Pekeris waveguide')
print('In this code, you will choose the time origin for warping')
print('Click once on the spectrogram at the position where you want to define the time origin')
print('')    
    


spectro_1=spectro[:1025,1300:1501]
time_s=time[0,1300:1501]
freq_s=freq[0,0:1025]

interactive(True)
point= pts_select(spectro_1,time_s,freq_s)
#point= pts_select(spectro)
print('When you have click to define the time origin, click "disconnect mpl" ') 

This is the spectrogram of a signal propagated in a Pekeris waveguide
In this code, you will choose the time origin for warping
Click once on the spectrogram at the position where you want to define the time origin



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have click to define the time origin, click "disconnect mpl" 


## 4. Shorten the signal, make it start at the chosen time origin, and warp it

In [5]:
# we extract the selected time origin 
pts=point.selected_points
pts_a=np.array(pts)
time_origin=pts_a[0,0]
time_t_dec=np.abs(time-time_origin)

ind0=np.where(time_t_dec==np.min(time_t_dec))
ind0=ind0[1]
ind0=ind0[0]
    
    
N_ok=150
s_ok=s_t[:,ind0:ind0+N_ok]
    
# Corresponding time and frequency axis
time_ok=np.arange(0,N_ok)/fs

#The warped signal will be s_w
[s_w, fs_w]=warp_temp_exa(s_ok,fs,r,c1)
M=len(s_w)

print('Continue')

Continue


## 5. Plot spectrograms

In [7]:
### Original signal
N_window=31  # you need a short window to see the modes

a=np.arange(1,N_ok+1)
a=a[np.newaxis,:]
d=np.hamming(N_window)
d=d[:,np.newaxis]
tfr=tfrstft(s_ok,a,NFFT,d)
spectro=abs(tfr)**2
        
           
### Warped signal
N_window_w=301  # You need a long window to see the warped modes
wind=np.hamming(N_window_w)
wind=wind/np.linalg.norm(wind)
wind=wind[:,np.newaxis]
b=np.arange(1,M+1)
b=b[np.newaxis,:]
tfr_w=tfrstft(s_w,b,NFFT,wind)
spectro_w=abs(tfr_w)**2

# Time and frequency axis of the warped signal
time_w=np.arange(0,(M)/fs_w, 1/fs_w)
freq_w=np.arange(0,fs_w-fs_w/NFFT+fs_w/NFFT,fs_w/NFFT)


# Figure

print('The left panel shows the spectrogram of the original signal with the chosen time origin')
print('The right panel shows the corresponding warped signal')
print('')

plt.figure(figsize=[8,6])
plt.subplot(121)
plt.imshow(spectro, extent=[time_ok[0,0], time_ok[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low')
plt.ylim([0, fs/2])
#plt.xlim([0, 0.5])  ### Adjust this to see better
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Original signal with chosen time origin')


plt.subplot(122)
plt.imshow(spectro_w, extent=[time_w[0], time_w[-1], freq_w[0], freq_w[-1]],aspect='auto',origin='low')
plt.ylim([0,40]) ### Adjust this to see better
plt.xlabel('Warped time (sec)')
plt.ylabel('Corresponding warped frequency (Hz)')
plt.title('Corresponding warped signal')
plt.show()

print('Run the previous cells from the point "3. Plot spectrogram and time origin selection" if you want to redo the time origin selection')
print('Continue if you want to proceed with modal filtering')

The left panel shows the spectrogram of the original signal with the chosen time origin
The right panel shows the corresponding warped signal



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Run the previous cells from the point "3. Plot spectrogram and time origin selection" if you want to redo the time origin selection
Continue if you want to proceed with modal filtering


## 6. Filtering

#### Filter mode 1

In [8]:
## selection the spectro_w part to mask 
spectro_w_1=spectro_w[0:400,:]
time_w_1=time_w
freq_w_1=freq_w[:400]
Nmode=4
modes=np.zeros((N_ok,Nmode))
tm=np.zeros((NFFT,Nmode))

# To make it easier, filtering will be done by hand using the roipoly tool.
# See python help for more information

print('Now try to filter the 4 modes');
print('Create the four masks sequentially, starting with mode 1, then 2, etc.')
print('To do so, click twice n the spectrogram to create a line and continue to define the region you want to filter ')
print('(if needed, go back to c_warping_and_filtering.m for mask creation)')
print('Look at Fig. 11 in the paper for a mask shape suggestion')
print('(you can enlarge the figure or make it full screen before creating the mask)')

# Let filter a mode 1
print('')
print('Filtering mode 1')
interactive(True)
section=bbox_select(spectro_w_1)
#section=bbox_select(spectro_w_1,time_w_1,freq_w_1)
print('When you have finish,click "disconnect mpl" and continue to filter mode 2') 

Now try to filter the 4 modes
Create the four masks sequentially, starting with mode 1, then 2, etc.
To do so, click twice n the spectrogram to create a line and continue to define the region you want to filter 
(if needed, go back to c_warping_and_filtering.m for mask creation)
Look at Fig. 11 in the paper for a mask shape suggestion
(you can enlarge the figure or make it full screen before creating the mask)

Filtering mode 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue to filter mode 2


#### Filter mode 2

In [9]:
# create the mask of the section
pts=section.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_1=np.double(mask)

# add the part masked to the total sprectogram array
masque_9=np.zeros_like(spectro_w[400:,:])
masque=np.concatenate((masque_1,masque_9),axis=0)


mode_rtf_warp=masque*tfr_w
norm=1/NFFT/np.max(wind)
mode_temp_warp=np.real(np.sum(mode_rtf_warp,axis=0))*norm*2
mode=iwarp_temp_exa(mode_temp_warp,fs_w,r,c1,fs,N_ok)
modes[:,0]=mode[:,0]

## Verification

a=hilbert(mode)
b=np.arange(1,N_ok+1)
b=b[np.newaxis,:]
h=np.hamming(N_window)
h=h[:,np.newaxis]
mode_stft=tfrstft(a,b,NFFT,h)
mode_spectro=abs(mode_stft)**2
tm_1,D2=momftfr(mode_spectro,0,N_ok,time_ok)
tm[:,0]=tm_1[:,0]

## Mode 2
print('')
print('Filtering mode 2')
interactive(True)
section_2=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue to filter mode 3') 


Filtering mode 2


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue to filter mode 3


#### Filter mode 3

In [10]:
# create the mask of the section
pts=section_2.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_2=np.double(mask)

# add the part masked to the total sprectogram array
masque_2=np.concatenate((masque_2,masque_9),axis=0)


mode_rtf_warp_2=masque_2*tfr_w
mode_temp_warp_2=np.real(np.sum(mode_rtf_warp_2,axis=0))*norm*2
mode_2=iwarp_temp_exa(mode_temp_warp_2,fs_w,r,c1,fs,N_ok)
modes[:,1]=mode_2[:,0]

## Verification
a_2=hilbert(mode_2)
mode_stft_2=tfrstft(a_2,b,NFFT,h)
mode_spectro_2=abs(mode_stft_2)**2
tm_2,D2_2=momftfr(mode_spectro_2,0,N_ok,time_ok)
tm[:,1]=tm_2[:,0]
## Mode 3
print('')
print('Filtering mode 3')
interactive(True)
section_3=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue to filter mode 4') 


Filtering mode 3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue to filter mode 4


#### Filter mode 4

In [11]:
# create the mask of the section
pts=section_3.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))

masque_3=np.double(mask)

# add the part masked to the total sprectogram array
masque_3=np.concatenate((masque_3,masque_9),axis=0)


mode_rtf_warp_3=masque_3*tfr_w
mode_temp_warp_3=np.real(np.sum(mode_rtf_warp_3,axis=0))*norm*2
mode_3=iwarp_temp_exa(mode_temp_warp_3,fs_w,r,c1,fs,N_ok)
modes[:,2]=mode_3[:,0]

## Verification

a_3=hilbert(mode_3)
mode_stft_3=tfrstft(a_3,b,NFFT,h)
mode_spectro_3=abs(mode_stft_3)**2
tm_3,D2_3=momftfr(mode_spectro_3,0,N_ok,time_ok)
tm[:,2]=tm_3[:,0]

## Mode 4

print('')
print('Filtering mode 4')
interactive(True)
section_4=bbox_select(spectro_w_1)
print('When you have finish,click "disconnect mpl" and continue') 


Filtering mode 4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Disconnect mpl', style=ButtonStyle())

When you have finish,click "disconnect mpl" and continue


In [12]:
# create the mask of the section
pts=section_4.selected_points
nx, ny = np.shape(spectro_w_1)
x, y = np.meshgrid(np.arange(ny), np.arange(nx))
x, y = x.flatten(), y.flatten()
points = np.vstack((x,y)).T
path = Path(pts)
grid = path.contains_points(points)
mask = grid.reshape((nx,ny))
masque_4=np.double(mask)

# add the part masked to the total sprectogram array
masque_4=np.concatenate((masque_4,masque_9),axis=0)


mode_rtf_warp_4=masque_4*tfr_w
mode_temp_warp_4=np.real(np.sum(mode_rtf_warp_4,axis=0))*norm*2
mode_4=iwarp_temp_exa(mode_temp_warp_4,fs_w,r,c1,fs,N_ok)
modes[:,3]=mode_4[:,0]

## Verification

a_4=hilbert(mode_4)
mode_stft_4=tfrstft(a_4,b,NFFT,h)
mode_spectro_4=abs(mode_stft_4)**2
tm_4,D2=momftfr(mode_spectro_4,0,N_ok,time_ok)
tm[:,3]=tm_4[:,0]

print('End of filtering')
print('Continue')


End of filtering
Continue


## 7. Verification

In [14]:
print('The red lines are the estimated dispersion curves.')
print(' ')

plt.figure(figsize=[7,5])
plt.imshow(spectro, extent=[time_ok[0,0], time_ok[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low')
plt.ylim([0,fs/2])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram and estimated dispersion curve')
plt.plot(tm[:,0],freq[0, :],'r')
plt.plot(tm[:,1],freq[0, :],'r')
plt.plot(tm[:,2],freq[0, :],'r')
plt.plot(tm[:,3],freq[0, :],'r')
plt.show()



print('Continue to look at your result vs the true modes')

The red lines are the estimated dispersion curves.
 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Continue to look at your result vs the true modes


## 8. Last verification

In [15]:
data = sio.loadmat(os.getcwd()+ '/sig_pek_and_modes_for_warp.mat')
r=data['r']
vg=data['vg']
c1=data['c1']
f_vg=data['f_vg']


### creation of the theoretical dispersion curves
tm_theo=r/vg-time_origin ### range over group_speed minus correction for time origin

print('This is the same figure than before, except that the true dispersion curves are now in black.')
print('How well did you do?')
print(' ')
print('Recall that you have to restrict interpretation of the dispersion curve (red line)')
print('to only the frequency band where it is relevant. The black and red lines will not match')
print('entirely for the length of the red line')
print(' ')

print('For practical applications, you will need to restrict dispersion curves to a frequency band')
print('where they are ok. This will be covered in the script g_filtering_multiple_modes_for_loc.')
print(' ')
print('Try to rerun the code and change the time origin to overly late/early value and see what it does.')
print(' ')


plt.figure(figsize=[7,5])
plt.imshow(spectro, extent=[time_ok[0,0], time_ok[0,-1], freq[0,0], freq[0,-1]],aspect='auto',origin='low')
plt.ylim([0,fs/2])
plt.xlim([0,0.74])
plt.xlabel('Time (sec)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram and estimated dispersion curve')
plt.plot(tm_theo[0,:], f_vg[0,:], 'black')
plt.plot(tm_theo[1,:], f_vg[0,:], 'black')
plt.plot(tm_theo[2,:], f_vg[0,:], 'black')
plt.plot(tm_theo[3,:], f_vg[0,:], 'black')
plt.plot(tm[:,0],freq[0, :],'r')
plt.plot(tm[:,1],freq[0, :],'r')
plt.plot(tm[:,2],freq[0, :],'r')
plt.plot(tm[:,3],freq[0, :],'r')
plt.show()


print(' ')
print('END')

This is the same figure than before, except that the true dispersion curves are now in black.
How well did you do?
 
Recall that you have to restrict interpretation of the dispersion curve (red line)
to only the frequency band where it is relevant. The black and red lines will not match
entirely for the length of the red line
 
For practical applications, you will need to restrict dispersion curves to a frequency band
where they are ok. This will be covered in the script g_filtering_multiple_modes_for_loc.
 
Try to rerun the code and change the time origin to overly late/early value and see what it does.
 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 
END
